In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd /content/drive/MyDrive/KOBERT

/content/drive/MyDrive/KOBERT


In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.7 MB/s eta 0:00:00


## TrainData
* 필요하면 Making Dataset 에서 불러오셈(CSV로 저장해 두었음)

## ValidData
* 필요하면 Making Dataset 에서 불러오셈(CSV로 저장해 두었음)

## DataFrame 구조로 불러오기 + 사용을 위한 전처리

In [ ]:
import pandas as pd
csv_file_path_t = '/content/drive/MyDrive/KOBERT/training.csv'
csv_file_path_v = '/content/drive/MyDrive/KOBERT/vailding.csv'
train_data= pd.read_csv(csv_file_path_t,encoding='utf-8',header=0)

test_data = pd.read_csv(csv_file_path_v,encoding='utf-8',header=0)
# DataFrame 출력


In [ ]:
%pip install sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(train_data, test_size=0.2)
# train_df = train_df.reset_index(drop=True)
# val_df = val_df.reset_index(drop=True)
train_data.shape, val_data.shape ,test_data.shape

((263289, 6), (65823, 6), (45144, 6))

In [ ]:
train_data.head()

,newsID,sent_num1,sent_num2,sentence1,sentence2,gold_label
160748,505272,3,4,"보도에 따르면 이동욱은 ‘풍선껌’에서 한방병원 원장 박리환 역을, 정려원은 라디오 ...",두 사람이 출연 예정인 ‘풍선껌’은 결핍을 가진 사람들이 각자의 빈틈을 품은 체 함...,Y
54197,632235,5,6,솜사탕 강민경은 “다비치로는 많이 노래를 들려드렸지만 강민경으로 들려드린 적은 많지...,이어 “가수로서 들은 칭찬 덕분에 용기가 생겼다”며 “노래보다 외모에 대한 이야기를...,Y
158473,505103,5,6,"이국주는 ""잠시 고민을 했지만 '출산의 고통보다 양악수술이 1000배 더 아팠다'는...","이어 ""2008년도 2월에는 정말 크게 헤어져 10개월 동안 만나지 못했다. 그런데...",N
256129,498663,3,4,첫 방송을 앞둔 대부분의 그룹은 긴장하기 마련. 그러나 아이콘은 '인기가요' 무대에...,"YG 엔터테인먼트 측 관계자는 일간스포츠에 ""멤버들이 추석 연휴까지 반납하고 컴백 ...",Y
288303,496444,4,6,지난 24일 공개된 강렬한 뮤직비디오 티저 영상을 통해 또 한 번 역대급 퍼포먼스를...,고객 맞춤형 컨설팅을 자랑하는 ‘내 이름은 캐디’는 24일 오후 9시에 안방극장에 ...,N


In [ ]:
from transformers import TFBertForMaskedLM
from transformers import AutoTokenizer

In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import urllib.request
from sklearn import preprocessing
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def drop_na_and_duplciates(df):
  df = df.dropna()
  df = df.drop_duplicates()
  df = df.reset_index(drop=True)
  return df

In [ ]:

# 결측값 및 중복 샘플제거
train_data = drop_na_and_duplciates(train_data)
val_data = drop_na_and_duplciates(val_data)
test_data = drop_na_and_duplciates(test_data)

# BERT 입력(Tokenizer 변경부분)

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

In [ ]:
max_seq_len = 256

In [ ]:
def convert_examples_to_features(sent_list1, sent_list2, max_seq_len, tokenizer):
  input_ids, attention_masks, token_type_ids = [], [], []
  for sent1, sent2 in tqdm(zip(sent_list1, sent_list2), total=len(sent_list1)):
    encoding_result = tokenizer.encode_plus(sent1, sent2, max_length=  max_seq_len, pad_to_max_length=True)
    input_ids.append(encoding_result['input_ids'])
    attention_masks.append(encoding_result['attention_mask'])
    token_type_ids.append(encoding_result['token_type_ids'])
  input_ids = np.array(input_ids, dtype=int)
  attention_masks = np.array(attention_masks, dtype=int)
  token_type_ids = np.array(token_type_ids, dtype=int)
  return (input_ids, attention_masks, token_type_ids)

In [ ]:
X_train = convert_examples_to_features(train_data['sentence1'], train_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 0/263289 [00:00<?, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
  0%|          | 1254/263289 [00:02<09:29, 460.07it/s]Be 

In [ ]:
X_val = convert_examples_to_features(val_data['sentence1'], val_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  3%|▎         | 2068/65823 [00:03<01:43, 614.40it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  6%|▌         | 4110/65823 [00:05<01:00, 1022.08it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have bee

In [ ]:
X_test = convert_examples_to_features(test_data['sentence1'], test_data['sentence2'], max_seq_len=max_seq_len, tokenizer=tokenizer)

  0%|          | 57/45144 [00:00<01:19, 565.65it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  1%|          | 423/45144 [00:00<01:12, 613.65it/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been re

In [ ]:
train_label = train_data['gold_label'].tolist()
val_label = val_data['gold_label'].tolist()
test_label = test_data['gold_label'].tolist()
idx_encode = preprocessing.LabelEncoder()
idx_encode.fit(train_label)
# 고유한       정수로       변환
y_train = idx_encode.transform(train_label)
y_val = idx_encode.transform(val_label)
y_test = idx_encode.transform(test_label)
label_idx = dict(zip(list(idx_encode.classes_), idx_encode.transform(list(idx_encode.classes_))))
##idx_encode.classes_ == c,e,n , idx_encode.classes_ == 0,1,2
idx_label = {value: key for key, value in label_idx.items()}
print('각  레이블과 정수:', label_idx)

각  레이블과 정수: {'N': 0, 'Y': 1}


In [ ]:
# 원본
# class TFBertForSequenceClassification(tf.keras.Model):
#   def __init__(self, model_name, num_labels):
#     super(TFBertForSequenceClassification, self).__init__()
#     self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
#     self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
#                                             activation='softmax',name='classifier')


#   def call(self, inputs):
#     input_ids, attention_mask, token_type_ids = inputs
#     outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
#     token_type_ids=token_type_ids)
#     cls_token = outputs[1]
#     prediction = self.classifier(cls_token)
#     return prediction

In [ ]:
# additional layer
class TFBertForSequenceClassification(tf.keras.Model):
  def __init__(self, model_name, num_labels):
    super(TFBertForSequenceClassification, self).__init__()
    self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
    self.classifier = tf.keras.layers.Dense(num_labels,kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                            activation='softmax',name='classifier')
    # 추가 된 것
    self.additional_layer = tf.keras.layers.Dense(
        128, #128 유닛을 가진 레이어 추가 (변경가능)
        activation ='relu'
    )


  def call(self, inputs):
    input_ids, attention_mask, token_type_ids = inputs
    outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask,
    token_type_ids=token_type_ids)
    cls_token = outputs[1]

    # 추가적인 레이어를 거쳐서 분류 수행
    hidden = self.additional_layer(cls_token)
    prediction = self.classifier(cls_token)
    return prediction

In [ ]:
# TPU 작  동  을       위  한       코  드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

## Bert모델 변경부분

In [ ]:
# # Create a new model instance
with strategy.scope():
  model = TFBertForSequenceClassification("klue/bert-base", num_labels=2)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(
  monitor="val_accuracy",
  min_delta=0.001,
  patience=2
)

# ModelCheckpoint 콜백 설정
checkpoint_path = '/content/drive/MyDrive/KOBERT/klue_check/kluecp_add.h5'  # 체크포인트 저장 경로::: h5 확장자로 만들어 주어야함.
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_path,  # 체크포인트 파일 저장 경로
    save_weights_only=True,    # 가중치만 저장
    monitor='val_accuracy',    # 모니터링할 지표 선택
    save_best_only=True,       # 최적 모델만 저장
    save_freq='epoch'          # 저장 주기 (에폭마다 저장)
)

# 모델 학습 및 콜백 적용
model.fit(X_train, y_train, epochs=2, batch_size=8, validation_data=(X_val, y_val),
          callbacks=[early_stopping, model_checkpoint])

Epoch 1/2


32912/32912 [==============================] - 3001s 88ms/step - loss: 0.1484 - accuracy: 0.9442 - val_loss: 0.1039 - val_accuracy: 0.9608
Epoch 2/2
32912/32912 [==============================] - 2922s 89ms/step - loss: 0.0907 - accuracy: 0.9675 - val_loss: 0.0896 - val_accuracy: 0.9687


In [ ]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(X_test, y_test, batch_size=8)[1]))

5643/5643 [==============================] - 180s 32ms/step - loss: 0.2437 - accuracy: 0.9195

 테스트 정확도: 0.9195


## 로드단계(여기서 실행 X)
* 여기서 부턴 다른 ipynb 에서 해야할 일임

In [ ]:
# from tensorflow.keras.models import load_model

# # 저장된 가중치를 로드하여 모델 생성
# loaded_model = TFBertForSequenceClassification("klue/bert-base", num_labels=2)
# loaded_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# loaded_model(X_train)  ##  예측할 데이터를 X_train 자리에 넣어주면 됨.
# loaded_model.load_weights(checkpoint_path)

# # 로드된 모델 사용
# predictions = loaded_model.predict(X_test, batch_size=8)


In [ ]:
# # 다른 페이지에서 활용한 예시
# from tensorflow.keras.models import load_model
# checkpoint_path = '/content/drive/MyDrive/KOBERT/klue_check/kluecp.h5'

# # 저장된 가중치를 로드하여 모델 생성
# loaded_model = TFBertForSequenceClassification("klue/bert-base", num_labels=2)
# loaded_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
# loaded_model(X1_test) # 예측하고 싶은 문장 넣으세요.
# loaded_model.load_weights(checkpoint_path)

# # 로드된 모델 사용
# predictions = loaded_model.predict(X1_test, batch_size=8)

In [ ]:
# predicted_classes = np.argmax(predictions, axis=1)

# print("Predicted classes:", predicted_classes)